# VGG
VGG16相比AlexNet的一个改进是采用连续的几个3x3的卷积核代替AlexNet中的较大卷积核（11x11，7x7，5x5）。对于给定的感受野（与输出有关的输入图片的局部大小），采用堆积的小卷积核是优于采用大的卷积核，因为多层非线性层可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。

## 导入库

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## 参数

In [2]:
BATCH_SIZE = 10000
EPOCH = 5
LR = 0.05

## 加载数据并调整大小

In [3]:
class DataLoader():
    def __init__(self):
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = keras.datasets.fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0, axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.train_images[index], 224, 224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.test_images[index], 224, 224,)
        return resized_images.numpy(), self.test_labels[index]

In [4]:
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)

## vgg卷积层

In [5]:
def vgg_block(num_convs, num_channels):
    blk = keras.models.Sequential()
    for _ in range(num_convs):
        blk.add(keras.layers.Conv2D(num_channels,kernel_size=3,padding='same',activation='relu'))
    blk.add(keras.layers.MaxPool2D(pool_size=2, strides=2))
    return blk

## 定义大模型

In [6]:
def vgg(conv_arch):
    model = keras.models.Sequential()
    for (num_convs, num_channels) in conv_arch:
        model.add(vgg_block(num_convs, num_channels))
    model.add(keras.models.Sequential([
        keras.layers.Flatten(),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4096, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(10, activation='sigmoid')
    ]))
    return model

In [7]:
CONV_ARCH = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))
model = vgg(CONV_ARCH)

## 定义优化器及损失

In [8]:
optimizer = keras.optimizers.SGD(learning_rate=LR)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 开始训练

In [9]:
num_iter = dataLoader.num_train//BATCH_SIZE
for e in range(EPOCH):
    for _ in range(num_iter):
        x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)
        model.fit(x_batch, y_batch)

Train on 10000 samples
10000/10000 [==============================] - 95s 10ms/sample - loss: 1.6625 - accuracy: 0.3339
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.7574 - accuracy: 0.7064
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.5580 - accuracy: 0.7922
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.4530 - accuracy: 0.8350
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.3969 - accuracy: 0.8518
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.3593 - accuracy: 0.8713
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.3350 - accuracy: 0.8743
Train on 10000 samples
10000/10000 [==============================] - 90s 9ms/sample - loss: 0.3145 - accuracy: 0.8835
Train on 10000 samples
10000/10000 [===========

## 测试

In [10]:
x_test, y_test = dataLoader.get_batch_test(2000)
model.evaluate(x_test, y_test)

2000/2000 - 6s - loss: 0.2316 - accuracy: 0.9185


[0.23162574183940887, 0.91850007]